# Playing BlackJack with First Visit MC

 First, let us import all the necessary libraries

In [2]:
import gym
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict
from functools import partial
%matplotlib inline
plt.style.use('ggplot')

 Simulate the Blackjack environment

In [11]:
#env = gym.make('Blackjack-v1', render_mode='human')
env = gym.make('Blackjack-v1')

In [15]:
import gym
import numpy as np
import random

class BlackjackEnv(gym.Env):
    def __init__(self):
        self.action_space = gym.spaces.Discrete(2)  # 0: stick, 1: hit
        self.observation_space = gym.spaces.Tuple((
            gym.spaces.Discrete(32),  # Player's current sum (0-31)
            gym.spaces.Discrete(11),  # Dealer's showing card (1-10)
            gym.spaces.Discrete(2)    # Whether player has a usable ace (0 or 1)
        ))
        self.reset()

    def step(self, action):
        if action:  # hit
            self.player_sum += self.draw_card()
            if self.player_sum > 21:
                done = True
                reward = -1
            else:
                done = False
                reward = 0
        else:  # stick
            done = True
            while self.dealer_sum < 17:
                self.dealer_sum += self.draw_card()
            reward = np.sign(self.player_sum - self.dealer_sum)
            if reward == 0:
                reward = 0
            elif reward > 0:
                reward = 1
            else:
                reward = -1
        return self._get_obs(), reward, done, {}

    def reset(self):
        self.player_sum = 0
        self.dealer_sum = 0
        self.player_sum += self.draw_card()
        self.dealer_sum += self.draw_card()
        self.player_sum += self.draw_card()
        self.dealer_sum += self.draw_card()
        return self._get_obs()

    def _get_obs(self):
        return (self.player_sum, self.dealer_sum, int(self.player_sum > 21))

    def draw_card(self):
        return min(random.randint(1, 13), 10)  # 1-10 are number cards, 10-13 are face cards

# Monte Carlo control algorithm
def monte_carlo_control(env, num_episodes, epsilon, decay_rate):
    # Get the sizes of the observation space and action space
    num_player_sum_values = env.observation_space[0].n
    print('num_player_sum_values'+ str(num_player_sum_values))
    num_dealer_showing_values = env.observation_space[1].n
    print('num_dealer_showing_values'+ str(num_dealer_showing_values))

    num_usable_ace_values = env.observation_space[2].n
    num_actions = env.action_space.n
    print('num_actions'+ str(num_actions))


    # Initialize Q-table and N-array
    Q = np.zeros((num_player_sum_values, num_dealer_showing_values, num_usable_ace_values, num_actions))
    N = np.zeros((num_player_sum_values, num_dealer_showing_values, num_usable_ace_values, num_actions))
    policy = np.ones((num_player_sum_values, num_dealer_showing_values, num_usable_ace_values, num_actions)) / num_actions

    # Initialize Q-table with dimensions based on the observation space and action space
    #Q = np.zeros((env.observation_space[0].n, env.observation_space[1].n, env.observation_space[2].n, env.action_space.n))
    #N = np.zeros((env.observation_space[0].n, env.observation_space[1].n, env.observation_space[2].n, env.action_space.n))
    #policy = np.ones((env.observation_space[0].n, env.observation_space[1].n, env.observation_space[2].n, env.action_space.n)) / env.action_space.n

    for episode in range(num_episodes):
        state = env.reset()
        episode_rewards = []
        while True:
            if random.random() < epsilon:
                action = env.action_space.sample()  # Explore
            else:
                action = np.argmax(policy[state])   # Exploit
            next_state, reward, done, _ = env.step(action)
            episode_rewards.append(reward)

            # Update counts
            N[state][action] += 1

            # Update Q-values
            alpha = 1 / N[state][action]
            Q[state][action] += alpha * (sum(episode_rewards) - Q[state][action])

            # Update policy
            policy[state] = epsilon / env.action_space.n
            best_action = np.argmax(Q[state])
            policy[state][best_action] += 1 - epsilon

            if done:
                break
            state = next_state

        # Decay epsilon
        epsilon *= decay_rate

    return policy

# Test the trained policy
def test_policy(env, policy, num_episodes):
    total_rewards = 0
    for _ in range(num_episodes):
        state = env.reset()
        while True:
            action = np.argmax(policy[state])
            state, reward, done, _ = env.step(action)
            total_rewards += reward
            if done:
                break
    average_reward = total_rewards / num_episodes
    return average_reward

# Parameters
num_episodes = 100000
epsilon = 0.5
decay_rate = 0.999

# Create Blackjack environment
env = BlackjackEnv()

# Train policy using Monte Carlo control
policy = monte_carlo_control(env, num_episodes, epsilon, decay_rate)

# Test trained policy
num_test_episodes = 10000
average_reward = test_policy(env, policy, num_test_episodes)
print(f"Average reward over {num_test_episodes} test episodes: {average_reward}")


num_player_sum_values32
num_dealer_showing_values11
num_actions2


IndexError: index 20 is out of bounds for axis 1 with size 11